In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from openai import OpenAI
import requests
import os
import torch
from dotenv import load_dotenv
import datasets
import pandas as pd
import datasets
import time
load_dotenv(override=True)

True

In [3]:
HUGGING_FACE_API_KEY = os.getenv("HUGGING_FACE_API_KEY")


In [4]:
model = AutoModelForCausalLM.from_pretrained(
            "apple/OpenELM-3B",
            trust_remote_code=True
        )

A new version of the following files was downloaded from https://huggingface.co/apple/OpenELM-3B:
- configuration_openelm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/apple/OpenELM-3B:
- modeling_openelm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
model.to(0)

OpenELMForCausalLM(
  (transformer): OpenELMModel(
    (token_embeddings): Embedding(32000, 3072)
    (layers): ModuleList(
      (0): OpenELMDecoderLayer(
        (attn): OpenELMMultiHeadCausalAttention(
          query_heads=12, key_heads=3, value_heads=3
          (qkv_proj): Linear(in_features=3072, out_features=2304, bias=False)
          (pos_embedding): OpenELMRotaryEmbedding(	model_dim=128, max_seq_length=4096, freq_constant=10000)
          (q_norm): OpenELMRMSNorm(num_features=128, eps=1e-06)
          (k_norm): OpenELMRMSNorm(num_features=128, eps=1e-06)
          (out_proj): Linear(in_features=1536, out_features=3072, bias=False)
        )
        (ffn): OpenELMFeedForwardNetwork(
          (ffn_with_glu) : True
          (proj_1): Linear(in_features=3072, out_features=3072, bias=False)
          (proj_2): Linear(in_features=1536, out_features=3072, bias=False)
          (act): SiLU()
        )
        (ffn_norm): OpenELMRMSNorm(num_features=3072, eps=1e-06)
        (attn_n

In [6]:
tokenizer = AutoTokenizer.from_pretrained(
            "meta-llama/Llama-2-7b-hf",
            trust_remote_code=True
        )

In [7]:
base_no_cot = """Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: The answer is 8.
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
A: The answer is 9.
Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?
A: The answer is 29.
Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?
A: The answer is 33.
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: The answer is 8."""

In [8]:
base_cot = """Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. The answer is 8.
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
A: Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. The answer is 9.
Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?
A: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The answer is 29.
Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?
A: Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. The answer is 33.
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8. The answer is 8."""

In [9]:
question = "Q: Albert is wondering how much pizza he can eat in one day. He buys 2 large pizzas and 2 small pizzas. A large pizza has 16 slices and a small pizza has 8 slices. If he eats it all, how many pieces does he eat that day?"

In [10]:
prompt = f"{base_no_cot}\nQ: {question}\n"

In [33]:
prompts = [prompt for i in range(10)]

In [11]:
tokenized_prompt = tokenizer(prompts)
tokenized_prompt = torch.tensor(
        tokenized_prompt['input_ids'],
        device=0
    )
tokenized_prompt = tokenized_prompt.to(0)


In [12]:
tokenized_prompt

tensor([[    1,   660, 29901,  ...,  2462, 29973,    13],
        [    1,   660, 29901,  ...,  2462, 29973,    13],
        [    1,   660, 29901,  ...,  2462, 29973,    13],
        ...,
        [    1,   660, 29901,  ...,  2462, 29973,    13],
        [    1,   660, 29901,  ...,  2462, 29973,    13],
        [    1,   660, 29901,  ...,  2462, 29973,    13]], device='cuda:0')

In [13]:
stime = time.time()
# batch

output_ids = model.generate(
    tokenized_prompt,
    max_length=1024,
    pad_token_id=0,
)
generation_time = time.time() - stime

In [23]:
output_ids

tensor([[    1,   660, 29901,  ...,  9292,  3287,   322],
        [    1,   660, 29901,  ...,  9292,  3287,   322],
        [    1,   660, 29901,  ...,  9292,  3287,   322],
        ...,
        [    1,   660, 29901,  ...,  9292,  3287,   322],
        [    1,   660, 29901,  ...,  9292,  3287,   322],
        [    1,   660, 29901,  ...,  9292,  3287,   322]], device='cuda:0')

In [35]:
output_text = tokenizer.batch_decode(
    output_ids,
    skip_special_tokens=True
)
output_text = [output[len(prompts[i]):].split("\n")[0] for i, output in enumerate(output_text)]
print("Generation time: ", generation_time)
print(output_text)


Generation time:  42.75608444213867
['A: The answer is 16.', 'A: The answer is 16.', 'A: The answer is 16.', 'A: The answer is 16.', 'A: The answer is 16.', 'A: The answer is 16.', 'A: The answer is 16.', 'A: The answer is 16.', 'A: The answer is 16.', 'A: The answer is 16.']
